# Capstone Project: Milestone Report
### Vikash Patel

As a large grocery and goods retailer, it is always in your best interests to hold your inventory at optimal levels. Store managers look to reduce spoilage risk of over-stocking perishable goods, and also hope to avoid being out-of-stock of goods that are heavily demanded. These scenarios are stressed during times of severe weather, such as heavy rainfall or snowfall. Are the effects of weather large enough to change buying patterns of potentially weather sensitive goods (like milk, bread, umbrellas) at stores? Or are there other factors that are more influential to consumers when going to the store? In this case study, I will be testing to see if there is a relationship between sales of weather sensitive goods and weather, and will also be forecasting future sales using a predictive regression model. The aim of this case study is to help give store managers a better approach on handling inventory management during times of severe weather.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

raw_weather = pd.read_csv('data/weather.csv')
sales_train = pd.read_csv('data/train.csv')
key = pd.read_csv('data/key.csv')

In [3]:
raw_weather.head(3)

,station_nbr,date,tmax,tmin,tavg,depart,dewpoint,wetbulb,heat,cool,sunrise,sunset,codesum,snowfall,preciptotal,stnpressure,sealevel,resultspeed,resultdir,avgspeed
0,1,2012-01-01,52,31,42,M,36,40,23,0,-,-,RA FZFG BR,M,0.05,29.78,29.92,3.6,20,4.6
1,2,2012-01-01,48,33,41,16,37,39,24,0,0716,1626,RA,0.0,0.07,28.82,29.91,9.1,23,11.3
2,3,2012-01-01,55,34,45,9,24,36,20,0,0735,1720,,0.0,0.00,29.77,30.47,9.9,31,10.0


In [4]:
sales_train.head(3)

,date,store_nbr,item_nbr,units
0,2012-01-01,1,1,0
1,2012-01-01,1,2,0
2,2012-01-01,1,3,0


In [5]:
key.head(3)

,store_nbr,station_nbr
0,1,1
1,2,14
2,3,7


The datasets I am working with are very basic in detail. There is no key as to what item each item number corresponds to. Also, a disclaimer in the sales dataset noted that item numbers are not consistent for each store, meaning item 1 in store 1 may not be the same as item 1 in store 2. This prevents me from looking at overall item trends across different stores in relation to weather. I will have to conduct this analysis on a store by store basis. The key dataset is what describes the relationship between certain stores and their respective weather stations, and will be necessary for joining the sales and weather data together. Multiple stores will point to the same weather station. From the weather dataset, we will be utilizing the 'tavg' (average daily temperature) and 'preciptotal' (daily precipitation total) columns, as both are primary indicators of the weather. Conditions with recorded precipitation and average temperatures below 32 degrees F will be indicative of winter weather (freezing rain, sleet, snow).